In [1]:
import pandas as pd
import numpy as np
eps = np.finfo(float).eps
from numpy import log2 as log

In [2]:
#entropy_node = 0

In [3]:
#def entropy(fraction):
#    entropy_node += -fraction*np.log2(fraction)

In [4]:
train = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_train.csv")
test = pd.read_csv(r"C:\Users\Ruchi\Documents\titanic_test.csv")

In [5]:
org_train = train.copy()
org_test = test.copy()

In [6]:
data = train.append(test, sort = True)

In [7]:
#data.info()
total_pass = 1309
print(data)

      age boat   body    cabin embarked      fare  \
0     NaN   13    NaN      NaN        Q    7.7333   
1    38.0  NaN    NaN      NaN        S    8.6625   
2    30.0  NaN    NaN      NaN        S   24.1500   
3    54.0    4    NaN      NaN        S   23.0000   
4    40.0  NaN    NaN      NaN        S   13.0000   
5    28.0  NaN  173.0      NaN        S   22.5250   
6    19.0  NaN    NaN      NaN        S    0.0000   
7    30.0   10    NaN      NaN        S   13.0000   
8    22.0    C    NaN      NaN        S    7.7750   
9    21.0  NaN    NaN      NaN        S    9.8250   
10   27.0  NaN    NaN      C82        C  211.5000   
11   60.0    8    NaN      D15        C   76.2917   
12   56.0    7    NaN      C50        C   83.1583   
13   20.0    4    NaN      NaN        S   23.0000   
14   16.0  NaN  190.0      NaN        S   20.2500   
15   48.0  NaN    NaN      NaN        S    7.8542   
16   40.0  NaN    NaN      NaN        S   16.0000   
17    NaN   16    NaN      NaN        Q   23.2

In [8]:
# Clean NA data
data.age.fillna(value = data.age.mean(), inplace = True)
data.fare.fillna(value = data.fare.mean(), inplace = True)
# Fill NA embarked values with the mode of embarked values
data.embarked.fillna(value=(data.embarked.value_counts().idxmax()), inplace=True)
data.survived.fillna(value=-1, inplace=True) 
data.boat.fillna(value = 0, inplace = True)
data['sex'] =  data['sex'].map( {'female': 0, 'male': 1} ).astype(int)
#Grouping ages
data.loc[data['age'] <= 16, 'age'] = 0
data.loc[(data['age'] > 16) & (data['age'] < 50), 'age'] = 1
data.loc[data['age'] >= 50, 'age'] = 2 

In [9]:
print(data)

     age boat   body    cabin embarked      fare  \
0    1.0   13    NaN      NaN        Q    7.7333   
1    1.0    0    NaN      NaN        S    8.6625   
2    1.0    0    NaN      NaN        S   24.1500   
3    2.0    4    NaN      NaN        S   23.0000   
4    1.0    0    NaN      NaN        S   13.0000   
5    1.0    0  173.0      NaN        S   22.5250   
6    1.0    0    NaN      NaN        S    0.0000   
7    1.0   10    NaN      NaN        S   13.0000   
8    1.0    C    NaN      NaN        S    7.7750   
9    1.0    0    NaN      NaN        S    9.8250   
10   1.0    0    NaN      C82        C  211.5000   
11   2.0    8    NaN      D15        C   76.2917   
12   2.0    7    NaN      C50        C   83.1583   
13   1.0    4    NaN      NaN        S   23.0000   
14   0.0    0  190.0      NaN        S   20.2500   
15   1.0    0    NaN      NaN        S    7.8542   
16   1.0    0    NaN      NaN        S   16.0000   
17   1.0   16    NaN      NaN        Q   23.2500   
18   1.0    

In [10]:
#data.drop('name', axis = 1)
#data = data.drop('sibsp', axis = 1)
data = data.drop(['name', 'sibsp', 'ticket', 'cabin', 'body','home.dest'], axis = 1)

In [11]:
data.head()

,age,boat,embarked,fare,parch,passenger_id,pclass,sex,survived
0,1.0,13,Q,7.7333,0,1216,3,0,1.0
1,1.0,0,S,8.6625,0,699,3,1,0.0
2,1.0,0,S,24.1500,1,1267,3,0,0.0
3,2.0,4,S,23.0000,3,449,2,0,1.0
4,1.0,0,S,13.0000,0,576,2,1,0.0


To generate a tree, calculate the entropy of your parent nodes.
Tree 1 : gender - age - survival boolean
Tree 2 : age - gender - survival boolean
Tree 3 : fare - gender - age


In [12]:
data['sex'].value_counts()
#Female : 0, Male : 1

1    843
0    466
Name: sex, dtype: int64

In [13]:
data['sex'].value_counts(1)

1    0.644003
0    0.355997
Name: sex, dtype: float64

In [14]:
male_ratio = 0.644003
female_ratio = 0.355997

1. Calculate the entropy of the target:

    Calculate entropy of survival (1 - survived, 0 - did not survive, -1 - NA)

In [15]:
e_node = 0  #Initialize Entropy
values = data['survived'].value_counts() #Unique objects - -1,0,+1
for value in values:
    fraction = data['survived'].value_counts()/len(data['survived'])  
    e_node += -fraction*np.log2(fraction)
print(e_node)

 0.0    1.582043
-1.0    1.590439
 1.0    1.480758
Name: survived, dtype: float64


In [16]:
print(data['survived'].value_counts())
print("Total passengers")
print(len(data['survived']))

 0.0    537
-1.0    459
 1.0    313
Name: survived, dtype: int64
Total passengers
1309


In [17]:
data['survived'].value_counts(1)

 0.0    0.410237
-1.0    0.350649
 1.0    0.239114
Name: survived, dtype: float64

In [18]:
died =  0.410237
na_survived =  0.350649
lived = 0.239114
e_survived = -(died*log(died)) -(na_survived*log(na_survived)) -(lived*log(lived))
print(e_survived)

1.5510801153857858


In [19]:
e_survived_no_na = -(died*log(died))-(lived*log(lived))
print(e_survived_no_na)

1.0209337235439513


Entropy having value > 1 just shows a very high level of disorder. This implies that we can't accurately predict the state of the passenger and that the result of the execution of this model will be more informative. 

In [20]:
# i being the information that number of dead gives us
# As the probability of a passenger not surviving is death_ratio < 1, these numbers give us high information
i = -log(died)
print(i)

1.285470477986138


As part of our first tree, we'll take Age as the parent node, split it to its different subgroups, which will be further split to sex, culminating in the survival of the passenger.

1. Get entropy of the parent as a whole
2. Get conditional entropy of subroups within parent
3. Find information gain by subtracting the weighted average of the conditional entropies from the parent's entropy

Example:
1. E(Age)
2. E(Age | Young), E(Age | Adult), E(Age | Old)
3. IG = E(Age) - E(weighted avg. conditional entropy)

In [21]:
data['age'].value_counts(1)

1.0    0.813598
0.0    0.102368
2.0    0.084034
Name: age, dtype: float64

In [22]:
e_age = -(0.813598*log(0.813598)) -(0.102368*log(0.102368)) - (0.084034*log(0.084034))
print(e_age)

0.87898284978781


In [23]:
e_age = 0  #Initialize Entropy
age_values = data['age'].value_counts() #Unique objects - -1,0,+1
for value in age_values:
    fraction = data['age'].value_counts()/len(data['age'])  
    e_age += -fraction*np.log2(fraction)
print(e_age)

1.0    0.726409
0.0    1.009809
2.0    0.900728
Name: age, dtype: float64


In [24]:
#Conditional entropy
#Young (0)| females(0) 
len(data.loc[(data['age'] == 0) & (data['sex'] == 0)])

64

In [25]:
#Young | Male
len(data.loc[(data['age'] == 0) & (data['sex'] == 1)])

70

In [26]:
num_young = len(data.loc[data['age'] == 0])
num_adult = len(data.loc[data['age'] == 1])
num_old = len(data.loc[data['age'] == 2])

In [27]:
print(num_old/total_pass)

0.08403361344537816


In [28]:
#Conditional entropy of Young(0) subgroup
#e_young = females/total young , males/total young
young_f = len(data.loc[(data['age'] == 0) & (data['sex'] == 0)])/num_young
young_m = len(data.loc[(data['age'] == 0) & (data['sex'] == 1)]) /num_young
e_young = -(young_f*log(young_f)) - (young_m*log(young_m))
print(e_young)

0.9985532860835362


In [29]:
#Conditional entropy of Adult (1) subgroup
adult_f = len(data.loc[(data['age'] == 1) & (data['sex'] == 0)]) / num_adult
adult_m = len(data.loc[(data['age'] == 1) & (data['sex'] == 1)]) / num_adult
e_adult = -(adult_f*log(adult_f)) -(adult_m*log(adult_m))
print(e_adult)

0.9265156836824285


In [30]:
#Conditional entropy of Old (2) subgroup
old_f = len(data.loc[(data['age'] == 2) & (data['sex'] == 0)]) / num_old
old_m = len(data.loc[(data['age'] == 2) & (data['sex'] == 1)]) / num_old
e_old = -(old_f*log(old_f)) -(old_m*log(old_m))
print(e_old)

0.9299429352091801


In [36]:
#Weighted avg entropy of age subgroups = total young / total passengers, total adult/ tot pass, total old/tot pass
ratio_y = num_young/total_pass
ratio_a = num_adult/total_pass
ratio_o = num_old/total_pass
e_age = -(ratio_y*log(ratio_y)) -(ratio_a*log(ratio_a))-(ratio_o*log(ratio_o))
print(e_age)

0.8789822416819767


In [37]:
e_wt_age = (ratio_y*e_young) + (ratio_a*e_adult) + (ratio_o*e_old)
print(e_wt_age)

0.9341780491443774


In [38]:
info_gain_age = e_age - e_wt_age
print(info_gain_age)

-0.055195807462400714
